<img src="resources/header.png"/>

# Population Assembly for BayesDB

Authored by: Veronica Weiner, Ph.D. and Ulrich Schaechtle, Ph.D. of the MIT Probabilistic Computing Project (Probcomp). Prepared for: Probcomp's workshop at the O'Reilly Media AI Conference, June 27, 2017, New York City.

## Outline

In this notebook, we will demonstrate how to construct BayesDB analysis populations -- consisting of **tabular, medium data** with defined statistical types -- from a variety of data sources. We provide examples of population assembly from: (1) a neuroscience data source of fMRI imaging data plus a separate public database of word meanings; (2) the Gapminder source for international development data; and (3) epidemiological data sources for tracking the spread of the flu based on Twitter and Centers for Disease Control data. The new skills to be demonstrated are in **bold** font below:

1. Introduction
    - 1.1 What is population assembly?
    - 1.2 A population is built from tabular data
    - 1.3 A population is built from medium data
    - 1.4 Variables in a population have defined statistical types
2. Launching BayesDB
    - 2.1 Setting up the Jupyter environment
    - 2.2 Creating a BayesDB `.bdb` file on disk
3. Assembling a population from two brain and cognitive science data sources, fMRI and word meanings
    - 3.1 Description of the data sources
    - 3.2 Ingesting data from a .csv file
    - 3.3 **Combining data from two different data sources into a single table**  
    - 3.4 **Letting BayesDB guess the statistical types of variables**
    - 3.5 **Creating the population**
4. Assembling a population from the Gapminder international development data source
    - 4.1 Description of the data source
    - 4.2 Ingesting data from a .csv file    
    - 4.3 **Nullifying missing values**
    - 4.4 **Reshaping data from a source by selecting a cross-section of it**
    - 4.5 **Creating new indicator variable**
    - 4.6 **Adding the new indicator variables to the data table**
    - 4.7 **Overriding BayesDB's default guesses for statistical types**
    - 4.8 Creating the population
5. Assembling a population from multiple epidemiological data sources for tracking the flu
    - 4.1 Description of the data sources
    - 4.2 **Creating a custom .csv file by using external Python code**
    - 4.3 Ingesting data from the .csv file
    - 4.4 Creating the population
6. Follow-up skills taught in other notebooks
    - 6.1 **Modeling and querying the populations you have created**

## 1. Introduction

### 1.1 What is population assembly?

BayesDB is a data science tool that, in its current form, is intended for use with **tabular, medium data**. This is data in two dimensions (rows and columns) that has between approximately 500 and 5,000,000 total cells (total cells = rows x columns). "Population assembly" is the process of reshaping the data from a data source of any shape, size, and contents into **tabular, medium data** that can be ingested by BayesDB. The process of population assembly also associates each variable in the data with a **statistical data type**, such as `NOMINAL`, `NUMERICAL`, `CYCLIC`, or `COUNTS`. 

The notion of a population is a central concept in BayesDB. Typically, a user assembles a population that is designed to probe a set of scientific or analytical questions of interest. Examples of data sources, from which populations can be assembled, include: 
* all measurements recorded from the Large Hadron Collider
* all biological brain data within the Allen Brain Atlas
* corporate financial data as time series for every public company in the United States
* measurements from the high-throughput wells of set of synthetic biology experiments 

### 1.2 A population is built from tabular data

Tabular data is data in a table: a two-dimensional format with rows and columns. A table used to build a BayesDB population has rows that represent observations, and columns that represent variables. Examples of tabular data include: 
* a table of earth-orbiting satellites (rows) and their characteristics like `"Country of Origin"`, `"Velocity"`, and `"Mass"` (columns)
* a table of patients (rows) and their health characteristics like `"Cytokines level"`, `"Age"`, and `"Has BRAC1 gene"` (columns)
* a table of sporting events (rows) and their characteristics like `"Date"`, `"Home team"`, `"Away team"`, and `"Score"` (columns)

Data from a large and/or multi-dimensional data source can be reshaped to create a two-dimensional data table. An example of multi-dimensional data is the [Gapminder dataset](http://www.gapminder.org/), described previously in this tutorial. The Gapminder dataset, originally assembled by Hans Rosling from the Gapminder Foundation, is a longitudinal dataset of ∼320 global indicators of socioeconomic development for 200 countries spanning more than 300 years. This data source includes time series (having two dimensions as `Measurement` and `Date`) and also identifies the `Country` for each. Data from this multi-dimensional source can be reshaped into numerous different two-dimensional tables depending on the question(s) of interest, for example by taking cross-sections or moving-window averages.

### 1.3 A population is built from medium data

Medium data, as distinguished from big data, includes typically tens (10's) to thousands (1000's) of columns, and tens (10's) to tens-of-thousands (10,000's) of rows. Many, if not the majority of, analytical questions of interest to scientists, professionals, and laypersons can be described as queries of a medium data population. For example, 75 petabytes of data were collected from the Large Hadron Collider experiments over three years, the largest scientific experiment in human history. However, the data were first filtered and reshaped to a medium data population of about 60,000 recorded events before estimating the mass of the Higgs Boson and publishing it as a scientific result.

### 1.4 Variables in a population have defined statistical types 

A BayesDB population includes one additional property. Each of the variables (columns) in an analysis population is associated with a statistical data type. Examples of statistical data types are `NOMINAL` for categorical data and `NUMERICAL` for real-valued data. Other types defined in BayesDB are `CYCLIC`, `COUNTS`, `MAGNITUDE`, and `NUMERICALRANGED`.

## 2. Launching BayesDB
### 2.1 Setting up the Jupyter environment


We first load the `jupyter_probcomp.magics` library. This is Probcomp's probabilistic programming environment that enables data exploration, plotting, querying, and analysis. It supports combinations of `SQL`, our BayesDB Bayesian Query Language (`BQL`), our Metamodeling Language (`MML`), our VentureScript language, plotting commands, and arbitrary Python code. The second cell configures plots from matplotlib and javascript to be shown inline.

In [1]:
%load_ext jupyter_probcomp.magics

session_id: fsaad@probcomp-1.csail.mit.edu_2017-12-17T10:51:11.306155_9


In [2]:
%matplotlib inline
%vizgpm inline

<IPython.core.display.Javascript object>

### 2.2 Creating a BayesDB `.bdb` file on disk

We use the `%bayesdb` magic command to create a `.bdb` file on disk named `population_assembly.bdb`. This file will store all the tables and populations created in this session.

In [3]:
!rm -f resources/population_assembly.bdb
%bayesdb resources/population_assembly.bdb

u'Loaded: resources/population_assembly.bdb'

## 3. Assembling a population from two brain and cognitive science data sources

### 3.1 Description of the data sources

This example shows how to construct a BayesDB population based on data from two distinct sources. The data sources here are (a) a brain imaging experiment, and (b) a publically available linguistic data set.

**(a) Brain imaging experimental data.** The first data source comprises recordings from a functional magnetic resonance imaging (fMRI) experiment in human subjects. Each subject was presented with 180 distinct words, and their brain activity was measured as fMRI voxel activations. A number of standard preprocessing steps were performed on the raw data by our collaborator before we received it (averaging over stimulus presentations, voxel selection using signal-to-noise, and projection of voxels into a lower-dimensional feature space). After receiving the fMRI data from our collaborator, for the purpose of this demonstration we used the data from one subject. The fMRI data is in the file `resources/fmri_features_100`.

**(b) Publicly available linguistic data.** The second data source includes information about word meaning. This includes a set of features, named GloVe vectors, that represent each word in a 300-dimensional semantic space. These word meaning features can be retrieved from a publicly available data source. We retrieved these features for the same 180 words in the fMRI experiment described above. For the purpose of this demonstration, the 300-dimensional semantic space was subsampled to 100 dimensions or word features. The word feature data is in the file `resources/word_features_100`.

**Collaborators.** The fMRI data for this example were provided by [Francisco Perreira](http://www.princeton.edu/~fpereira/index.shtml) of Siemens Healthcare, previously of Princeton University where he assisted in collecting the data. GloVe Vectors are those in the public dataset from Christopher Manning's group at Stanford (https://nlp.stanford.edu/projects/glove/) and were also provided to us by Francisco Perreira.

### 3.2 Ingesting data from a .csv file

We begin by creating `SQL` tables from the `.csv` data files. The first file contains fMRI data: `resources/fmri_features_100.csv`. The second file contains the word meaning data: `resources/word_features_100.csv`.  Each `.csv` file has 180 rows corresponding to the 180 words in the experiment. Each `.csv` file has 100 columns as described above.

#### (a) Brain imaging experimental data

The following cell constructs a `SQL` table from the first `.csv` file of brain imaging (fMRI) data.

In [4]:
%bql DROP TABLE IF EXISTS fmri_feature_data_table;
%bql CREATE TABLE fmri_feature_data_table FROM 'resources/fmri_features_100.csv'

""


Now we display the results, showing 5 rows from the `SQL` table we have just created.

In [5]:
%sql SELECT * FROM fmri_feature_data_table LIMIT 5;

,word,fmri_feature_11,fmri_feature_170,fmri_feature_218,fmri_feature_202,fmri_feature_160,fmri_feature_175,fmri_feature_162,fmri_feature_84,fmri_feature_163,fmri_feature_145,fmri_feature_9,fmri_feature_191,fmri_feature_55,fmri_feature_236,fmri_feature_30,fmri_feature_90,fmri_feature_56,fmri_feature_196,fmri_feature_221,fmri_feature_10,fmri_feature_244,fmri_feature_120,fmri_feature_152,fmri_feature_112,fmri_feature_165,fmri_feature_225,fmri_feature_255,fmri_feature_64,fmri_feature_67,fmri_feature_23,fmri_feature_177,fmri_feature_266,fmri_feature_223,fmri_feature_247,fmri_feature_151,fmri_feature_291,fmri_feature_142,fmri_feature_88,fmri_feature_259,fmri_feature_199,fmri_feature_3,fmri_feature_217,fmri_feature_119,fmri_feature_73,fmri_feature_110,fmri_feature_121,fmri_feature_258,fmri_feature_231,fmri_feature_72,fmri_feature_66,fmri_feature_135,fmri_feature_185,fmri_feature_176,fmri_feature_83,fmri_feature_284,fmri_feature_61,fmri_feature_183,fmri_feature_299,fmri_feature_208,fmri_feature_290,fmri_feature_79,fmri_feature_85,fmri_feature_102,fmri_feature_254,fmri_feature_201,fmri_feature_277,fmri_feature_237,fmri_feature_234,fmri_feature_157,fmri_feature_200,fmri_feature_45,fmri_feature_108,fmri_feature_117,fmri_feature_289,fmri_feature_71,fmri_feature_239,fmri_feature_8,fmri_feature_267,fmri_feature_94,fmri_feature_139,fmri_feature_74,fmri_feature_100,fmri_feature_68,fmri_feature_125,fmri_feature_86,fmri_feature_173,fmri_feature_161,fmri_feature_15,fmri_feature_260,fmri_feature_180,fmri_feature_105,fmri_feature_13,fmri_feature_212,fmri_feature_187,fmri_feature_116,fmri_feature_75,fmri_feature_128,fmri_feature_5,fmri_feature_52,fmri_feature_2
0,ability,20.451880,13.604858,23.854502,15.959425,51.567362,41.337091,26.618388,18.438101,39.517873,23.477500,46.525425,8.252563,27.147015,38.680716,31.995819,5.456450,43.756981,50.828288,23.555348,37.775880,31.577228,64.151345,20.324630,41.388384,17.552648,23.869646,33.395534,50.421412,14.297934,55.580607,15.437354,37.084569,64.951590,8.274478,49.078021,7.720060,6.789415,34.089867,24.148597,26.027394,24.265454,32.449634,36.029195,6.454516,19.150673,35.165447,54.510134,25.220418,9.768588,32.072957,38.273556,16.303695,27.829222,25.607912,26.802108,21.424317,17.337094,48.129923,19.227211,25.846847,17.746263,24.525755,12.927321,7.246685,9.356816,19.137917,57.784747,20.358349,24.571857,2.106094,34.322874,42.902952,25.566246,15.095134,42.557857,30.862065,35.739210,9.056162,6.759433,14.844401,-2.454790,31.037415,24.926361,10.226871,31.610405,56.744429,20.858573,29.238161,-7.456525,45.579007,43.627221,20.261596,35.173176,28.723186,20.836330,22.084941,-26.662809,7.861713,42.325707,25.326793
1,accomplished,2.766129,25.198853,34.543838,41.377115,33.606568,9.368161,28.945849,32.615977,49.850563,38.567062,39.303211,40.516346,47.407183,67.944972,37.152247,24.074907,34.075607,34.383562,20.447536,-11.374169,47.586585,49.562638,28.425691,58.103539,16.612205,28.060753,30.351340,40.085390,37.351457,37.518591,45.951910,54.648577,15.033508,45.519151,25.716890,6.378096,24.138598,47.460326,57.225440,28.348595,21.889621,22.840010,29.392581,45.797751,33.888349,15.731089,40.736132,32.369390,28.012996,19.634133,11.298614,29.398275,43.931314,31.287699,25.836469,61.081322,29.957176,27.045158,21.191186,11.190760,19.342883,38.862699,41.985273,26.505201,36.468753,31.993024,12.860892,29.787856,1.597974,36.676273,5.747011,43.358543,12.546523,65.437312,40.184492,21.447430,52.694001,52.596052,19.168592,38.225344,32.253297,24.717128,5.969136,11.436989,21.597122,22.711005,18.902236,39.295764,13.087015,-4.987606,48.234932,31.991038,25.573872,49.982572,9.863009,-5.096981,13.527659,28.556824,-1.517992,12.980697
2,angry,32.778588,13.763113,12.753896,16.471692,37.183198,31.794906,0.506357,51.230994,36.178480,33.144896,28.808484,29.952168,23.399445,20.650556,51.033644,12.443760,18.553432,31.596673,29.868056,18.161174,27.604546,28.017154,24.879160,55.640016,25.688473,22.802721,26.448740,34.133386,-4.757533,24.665161,16.973801,50.36170

#### (b) Publicly available linguistic data

The following cell constructs a `SQL` table from the second `.csv` file of linguistic data (GloVe word features).

In [6]:
%bql DROP TABLE IF EXISTS word_feature_data_table;
%bql CREATE TABLE word_feature_data_table FROM 'resources/word_features_100.csv'

""


Now we display the results, showing 5 rows from the `SQL` table we have just created.

In [7]:
%sql SELECT * FROM word_feature_data_table LIMIT 5;

,word,word_feature_264,word_feature_35,word_feature_292,word_feature_20,word_feature_12,word_feature_284,word_feature_165,word_feature_266,word_feature_139,word_feature_157,word_feature_281,word_feature_56,word_feature_294,word_feature_193,word_feature_162,word_feature_119,word_feature_118,word_feature_299,word_feature_236,word_feature_231,word_feature_221,word_feature_289,word_feature_223,word_feature_222,word_feature_163,word_feature_140,word_feature_11,word_feature_108,word_feature_42,word_feature_248,word_feature_16,word_feature_125,word_feature_152,word_feature_44,word_feature_107,word_feature_258,word_feature_275,word_feature_52,word_feature_124,word_feature_72,word_feature_195,word_feature_290,word_feature_297,word_feature_121,word_feature_126,word_feature_142,word_feature_155,word_feature_237,word_feature_138,word_feature_96,word_feature_296,word_feature_128,word_feature_235,word_feature_150,word_feature_242,word_feature_175,word_feature_33,word_feature_230,word_feature_254,word_feature_257,word_feature_76,word_feature_220,word_feature_226,word_feature_159,word_feature_212,word_feature_213,word_feature_172,word_feature_26,word_feature_168,word_feature_143,word_feature_67,word_feature_280,word_feature_190,word_feature_130,word_feature_80,word_feature_27,word_feature_86,word_feature_244,word_feature_63,word_feature_94,word_feature_300,word_feature_129,word_feature_277,word_feature_205,word_feature_176,word_feature_239,word_feature_291,word_feature_188,word_feature_65,word_feature_232,word_feature_214,word_feature_7,word_feature_110,word_feature_100,word_feature_144,word_feature_4,word_feature_167,word_feature_206,word_feature_216,word_feature_171
0,ability,0.176980,0.043515,0.062159,0.114000,-0.397430,0.022635,0.173930,0.009311,-0.280130,0.06611,0.191980,-0.134400,0.18498,0.31676,0.337310,0.033535,0.23530,0.137870,-0.17678,-0.26941,-0.33435,-0.130020,0.203480,0.301350,-0.123030,-0.051691,-0.183090,0.20621,-0.41150,-0.005104,-0.047276,-0.11098,0.287800,-0.202130,0.356140,0.200480,0.079314,-0.176590,0.12215,-0.209710,0.19767,0.066200,0.762380,-0.314180,0.135650,-0.211110,0.179230,0.24203,-0.012086,0.040704,0.115120,-0.001759,-0.033794,0.118780,0.298650,-0.092322,-0.043477,0.348930,0.362830,0.044559,-0.544600,-0.037531,0.249850,0.003210,0.42626,-0.64857,-0.28090,-0.295990,-0.042187,-0.399460,0.62705,0.48203,-0.29888,-0.348150,0.050691,-0.157580,-0.342430,-0.30516,-0.18161,0.15052,0.42464,0.292750,-0.461830,0.008585,0.40893,0.30700,0.023627,0.089886,0.072492,0.066342,-0.20825,0.38266,0.348410,0.34870,0.031341,-0.015382,0.001184,0.069693,-0.231110,0.089517
1,accomplished,-0.119390,-0.702120,-0.406910,0.299480,0.007565,0.187570,0.121810,0.159090,-0.270980,-0.17135,0.034012,-0.025162,-0.29928,-0.40455,-0.192230,0.398860,-0.31176,-0.231700,-0.18042,0.38330,0.17434,-0.215300,-0.210530,0.219150,-0.708860,0.164660,0.067617,0.69312,0.41542,0.153030,0.267050,0.16776,0.009577,-0.299530,-0.208450,-0.353820,0.131250,0.311220,0.39103,0.091806,0.24261,-0.107050,0.302320,-0.043134,-0.353290,0.164780,0.073025,0.66524,0.140260,-0.261840,-0.096772,-0.454050,-0.054707,-0.307920,-0.146040,-0.542220,-0.279020,0.034395,0.185750,-0.351810,-0.223730,-0.314050,-0.067944,-0.179610,-0.33105,-0.25749,0.15464,-0.045033,-0.021135,-0.214330,-0.08239,-0.30467,-0.34231,-0.018828,-0.310750,0.472140,-0.003174,-0.36677,0.25305,0.14988,-0.21295,0.056029,-0.060098,0.348090,0.34368,0.36037,0.075995,0.179070,-0.194060,0.416720,-0.11254,0.55969,-0.146850,0.54239,0.000897,-0.150200,0.046792,-0.160920,0.163280,-0.221880
2,angry,-0.171010,-0.196760,-0.241950,0.069393,0.093892,-0.261890,0.004182,-0.107850,0.309070,0.17340,0.070579,0.273040,0.13990,0.38807,-0.034515,-0.146220,-0.15737,0.101600,-0.18091,-0.62083,0.39780,-0.100700,0.272940,-0.401030,-0.164200,0.230790,0.046562,-0.29183,-0.12373,-0.353980,0.553180,0.15865,0.373140,0.393700,-0.323330,0.378360,-0.128220,-0.421470,0.19300,0.410540,0.33440,0.256980,0.309510,0.273810,-0.489820,0.080352,-0.069918,0.11858,-0.2424

### 3.3 Combining data from two different data sources into a single table 

Next, we create a table that combines the fMRI features and word features using the following `SQL` expression. Afterward, we display the results.

In [8]:
%%sql 
DROP TABLE IF EXISTS combined_word_and_fmri_feature_table;

CREATE TABLE combined_word_and_fmri_feature_table AS
    SELECT * FROM fmri_feature_data_table, word_feature_data_table 
    WHERE fmri_feature_data_table.word = word_feature_data_table.word;

""


In [9]:
%sql SELECT * FROM combined_word_and_fmri_feature_table LIMIT 5;

,word,fmri_feature_11,fmri_feature_170,fmri_feature_218,fmri_feature_202,fmri_feature_160,fmri_feature_175,fmri_feature_162,fmri_feature_84,fmri_feature_163,fmri_feature_145,fmri_feature_9,fmri_feature_191,fmri_feature_55,fmri_feature_236,fmri_feature_30,fmri_feature_90,fmri_feature_56,fmri_feature_196,fmri_feature_221,fmri_feature_10,fmri_feature_244,fmri_feature_120,fmri_feature_152,fmri_feature_112,fmri_feature_165,fmri_feature_225,fmri_feature_255,fmri_feature_64,fmri_feature_67,fmri_feature_23,fmri_feature_177,fmri_feature_266,fmri_feature_223,fmri_feature_247,fmri_feature_151,fmri_feature_291,fmri_feature_142,fmri_feature_88,fmri_feature_259,fmri_feature_199,fmri_feature_3,fmri_feature_217,fmri_feature_119,fmri_feature_73,fmri_feature_110,fmri_feature_121,fmri_feature_258,fmri_feature_231,fmri_feature_72,fmri_feature_66,fmri_feature_135,fmri_feature_185,fmri_feature_176,fmri_feature_83,fmri_feature_284,fmri_feature_61,fmri_feature_183,fmri_feature_299,fmri_feature_208,fmri_feature_290,fmri_feature_79,fmri_feature_85,fmri_feature_102,fmri_feature_254,fmri_feature_201,fmri_feature_277,fmri_feature_237,fmri_feature_234,fmri_feature_157,fmri_feature_200,fmri_feature_45,fmri_feature_108,fmri_feature_117,fmri_feature_289,fmri_feature_71,fmri_feature_239,fmri_feature_8,fmri_feature_267,fmri_feature_94,fmri_feature_139,fmri_feature_74,fmri_feature_100,fmri_feature_68,fmri_feature_125,fmri_feature_86,fmri_feature_173,fmri_feature_161,fmri_feature_15,fmri_feature_260,fmri_feature_180,fmri_feature_105,fmri_feature_13,fmri_feature_212,fmri_feature_187,fmri_feature_116,fmri_feature_75,fmri_feature_128,fmri_feature_5,fmri_feature_52,fmri_feature_2,word:1,word_feature_264,word_feature_35,word_feature_292,word_feature_20,word_feature_12,word_feature_284,word_feature_165,word_feature_266,word_feature_139,word_feature_157,word_feature_281,word_feature_56,word_feature_294,word_feature_193,word_feature_162,word_feature_119,word_feature_118,word_feature_299,word_feature_236,word_feature_231,word_feature_221,word_feature_289,word_feature_223,word_feature_222,word_feature_163,word_feature_140,word_feature_11,word_feature_108,word_feature_42,word_feature_248,word_feature_16,word_feature_125,word_feature_152,word_feature_44,word_feature_107,word_feature_258,word_feature_275,word_feature_52,word_feature_124,word_feature_72,word_feature_195,word_feature_290,word_feature_297,word_feature_121,word_feature_126,word_feature_142,word_feature_155,word_feature_237,word_feature_138,word_feature_96,word_feature_296,word_feature_128,word_feature_235,word_feature_150,word_feature_242,word_feature_175,word_feature_33,word_feature_230,word_feature_254,word_feature_257,word_feature_76,word_feature_220,word_feature_226,word_feature_159,word_feature_212,word_feature_213,word_feature_172,word_feature_26,word_feature_168,word_feature_143,word_feature_67,word_feature_280,word_feature_190,word_feature_130,word_feature_80,word_feature_27,word_feature_86,word_feature_244,word_feature_63,word_feature_94,word_feature_300,word_feature_129,word_feature_277,word_feature_205,word_feature_176,word_feature_239,word_feature_291,word_feature_188,word_feature_65,word_feature_232,word_feature_214,word_feature_7,word_feature_110,word_feature_100,word_feature_144,word_feature_4,word_feature_167,word_feature_206,word_feature_216,word_feature_171
0,ability,20.451880,13.604858,23.854502,15.959425,51.567362,41.337091,26.618388,18.438101,39.517873,23.477500,46.525425,8.252563,27.147015,38.680716,31.995819,5.456450,43.756981,50.828288,23.555348,37.775880,31.577228,64.151345,20.324630,41.388384,17.552648,23.869646,33.395534,50.421412,14.297934,55.580607,15.437354,37.084569,64.951590,8.274478,49.078021,7.720060,6.789415,34.089867,24.148597,26.027394,24.265454,32.449634,36.029195,6.454516,19.150673,35.165447,54.510134,25.220418,9.768588,32.072957,38.273556,16.303695,27.829222,25.607912,26.802108,21.424317,17.337094,48.129923,19.227211,25.846847,17.746263,24.525755,12.927321,7.246685,9.356816

### 3.4 Letting BayesDB guess the statistical types of variables

Now that we have created a data table, we can create a BayesDB population from it. A population associates each variable with a statistical data type (such as `NOMINAL` or `NUMERICAL`), and specifies non-probabilistic variables which should be ignored. 

BayesDB can guess statistical types automatically. BayesDB guesses (i) which variables to ignore, and (ii) the statistical data types of all other variables. Using the expression in the next cell, we first confirm that BayesDB's guesses are reasonable. BayesDB has guessed that every column (variable) here is `NUMERICAL`. This is correct: the fMRI activation features and word features both include real-valued data. BayesDB has also ignored the variable `"word"` from both original tables. This is also correct, because `"word"` is a non-probabilistic label rather than a numerical or categorical variable.

In [10]:
%mml .guess_schema combined_word_and_fmri_feature_table

MODEL 
	 "word_feature_235",
	 "fmri_feature_128",
	 "word_feature_237",
	 "fmri_feature_68",
	 "word_feature_231",
	 "word_feature_230",
	 "fmri_feature_201",
	 "fmri_feature_121",
	 "fmri_feature_120",
	 "fmri_feature_61",
	 "fmri_feature_67",
	 "fmri_feature_66",
	 "fmri_feature_64",
	 "word_feature_107",
	 "fmri_feature_73",
	 "fmri_feature_221",
	 "word_feature_100",
	 "word_feature_20",
	 "word_feature_27",
	 "word_feature_26",
	 "word_feature_239",
	 "fmri_feature_75",
	 "word_feature_242",
	 "word_feature_244",
	 "word_feature_248",
	 "word_feature_275",
	 "fmri_feature_79",
	 "word_feature_175",
	 "word_feature_4",
	 "word_feature_7",
	 "fmri_feature_71",
	 "fmri_feature_72",
	 "fmri_feature_157",
	 "fmri_feature_74",
	 "fmri_feature_151",
	 "fmri_feature_152",
	 "fmri_feature_231",
	 "word_feature_138",
	 "word_feature_139",
	 "fmri_feature_234",
	 "fmri_feature_237",
	 "fmri_feature_236",
	 "fmri_feature_239",
	 "word_feature_130",
	 "word_feature_33",
	 "word_feature_35",
	

### 3.5 Creating the population

Having confirmed that BayesDB correctly guessed the statistical types for `combined_word_and_fmri_feature_table`, we create a new population from that table with the expression below. By convention, we name the population the same as the original table but with `"population"` as the suffix. After a BayesDB population is created, it can be used for modeling, analysis, and querying in BayesDB. 

In [11]:
%%mml
DROP POPULATION IF EXISTS combined_word_and_fmri_feature_population;

CREATE POPULATION combined_word_and_fmri_feature_population
    FOR combined_word_and_fmri_feature_table WITH SCHEMA ( GUESS STATTYPES OF (*) );

""


## 4. Assembling a population from the Gapminder international development data source

### 4.1 Description of the data source

The next data source is published by the [Gapminder project](http://www.gapminder.org/). This is an international effort to collect data related to socioeconomic development from about 200 countries, spanning 300+ years. We demonstrate how to assemble an example population from this multi-dimensional data source. First, we show how `SQL` can be used to select a cross-section of the data, here selecting data for one year only. Second, we investigate the fact that many of the entries in this data table are missing (`NaN`). We show how to create a new indicator variable to denote cells with missing data. Then we construct a new table with these "missingness indicators" added, and create a population from that table. This analysis popuation can be used for some interesting analyses, for example to assess whether missing values can safely be treated as random, or are missing not at random (MNAR). 

### 4.2 Ingesting data from a .csv file

We ingest data from a `.csv` file into a `SQL` data table as we have demonstrated earlier in this notebook.

In [12]:
%bql DROP TABLE IF EXISTS gapminder_raw_data_table;
%bql CREATE TABLE gapminder_raw_data_table FROM 'resources/country-year.csv';

""


We display the contents of 5 rows of the new Gapminder data table.

In [13]:
%bql SELECT * FROM gapminder_raw_data_table LIMIT 5

,country_id,country,continent,year,aid_received,aid_received_log,invest_foreign_per1k,invest_foreign_per1k_log,invest_domestic_per1k,invest_domestic_per1k_log,spending_health_per1,spending_health_per1_log,gdp_per1k,mortality_kid,mortality_kid_log,life_expectancy,physicians,population,population_density,surface,gini,agriculture,hiv,malaria,professional_birth,sugar,food,infection,labor,broadband,cell_phone,internet,computers,it_tel,drought,drought_log,earthquake,earthquake_log,epidemic,epidemic_log,extreme_temp,extreme_temp_log,flood,flood_log,storm,storm_log,tsunami,tsunami_log,air_accident,air_accident_log,immune_diphtheria,immune_hepatitis,immune_hib,mcv,immune_tetanus,coal_per1,coal_per1_log,natural_gas_per1,natural_gas_per1_log,co2_per1,co2_per1_log,completion,energy_per1,energy_per1_log,sanitation,water_source,electricity_per1,electricity_per1_log,oil_per1,oil_per1_log,mortality_maternal,gdp_per1k_log,life_expectancy_log,mortality_maternal_log,delta_aid_received_log,delta_invest_foreign_per1k_log,delta_invest_domestic_per1k_log,delta_spending_health_per1_log,delta_gdp_per1k_log,delta_mortality_kid_log,delta_life_expectancy_log,delta_mortality_maternal_log,delta_aid_received_log_lag1,delta_aid_received_log_lag2,delta_aid_received_log_lag3,delta_aid_received_log_lag4,delta_invest_foreign_per1k_log_lag1,delta_invest_foreign_per1k_log_lag2,delta_invest_foreign_per1k_log_lag3,delta_invest_foreign_per1k_log_lag4,delta_invest_domestic_per1k_log_lag1,delta_invest_domestic_per1k_log_lag2,delta_invest_domestic_per1k_log_lag3,delta_invest_domestic_per1k_log_lag4,delta_spending_health_per1_log_lag1,delta_spending_health_per1_log_lag2,delta_spending_health_per1_log_lag3,delta_spending_health_per1_log_lag4
0,2,Afghanistan,Asia,1960,108800000,8.036629,NA,NA,0.194355,-0.711403,NA,NA,1.206,362.4,2.559188,35.68460,0.0348442,8994793,14.831,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000046,-4.336641,NA,NA,NA,NA,NA,NA,NA,NA,NA,369.938,0.081347,1.552481,2.56813,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,2,Afghanistan,Asia,1961,223030000,8.348363,NA,NA,0.197862,-0.703637,NA,NA,1.192,356.5,2.552060,36.30334,NA,9164945,15.121,652230,NA,57.8017,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000054,-4.270754,NA,NA,NA,NA,NA,NA,NA,NA,NA,992.396,0.076276,1.559947,2.99668,0.311734,NA,0.00776662,NA,-0.00507105,-0.00712865,0.00746575,0.428556,8.03663,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,2,Afghanistan,Asia,1962,110970000,8.045206,NA,NA,0.226976,-0.644021,NA,NA,1.188,350.6,2.544812,36.92308,NA,9343772,15.422,652230,NA,57.8937,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000074,-4.132093,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.074816,1.567298,NA,-0.303158,NA,0.0596161,NA,-0.00145981,-0.00724762,0.00735134,NA,8.34836,8.03663,NA,NA,NA,NA,NA,NA,0.00776662,NA,NA,NA,NA,NA,NA,NA
3,2,Afghanistan,Asia,1963,225130000,8.352433,NA,NA,0.168284,-0.773957,NA,NA,1.185,345.0,2.537819,37.54182,NA,9531555,15.736,652230,NA,57.9703,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000074,-4.129335,NA,NA,NA,NA,NA,NA,NA,NA,NA,1976.18,0.073718,1.574515,3.29583,0.307228,NA,-0.129936,NA,-0.00109809,-0.00699282,0.0072174,NA,8.04521,8.34836,8.03663,NA,NA,NA,NA,NA,0.0596161,0.00776662,NA,NA,NA,NA,NA,NA
4,2,Afghanistan,Asia,1964,290570000,8.463251,NA,NA,0.164167,-0.784715,NA,NA,1.182,339.7,2.531096,38.15956,NA,9728645,16.064,652230,NA,58.0669,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000086,-4.063945,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.072617,1.581603,NA,0.110817,NA,-0.0107579,NA,-0.00110087,-0.00672355,0.00708804,NA,8.35243,8.04521,8.34836,8.03663,NA,NA,NA,NA,-0.129936,0.0596161,0.00776662,NA,NA,NA,NA,NA


Using `SQL`, we can examine the contents of the table. For example, we display that it has 8670 rows in total:

In [14]:
%bql SELECT COUNT(*) FROM gapminder_raw_data_table

,"""COUNT""(*)"
0,8670


### 4.3 Nullifying missing values

Almost all datasets have missing values that are denoted by special tokens like `NaN`, `NA`, or an empty cell in the `.csv` file. We see above some of the table cells denote missing data using the token 'NA'. Let's tell the BayesDB system to consider those as missing and treat them as a SQL `NULL` value. We do this using the `bql` command `.nullify`. More than 300,000 cells have been nullified, indicating that this data is quite sparse.

In [15]:
%bql .nullify gapminder_raw_data_table 'NA'

Nullified 373148 cells


### 4.4 Reshaping data from a source by selecting a cross-section of it

Assembling a population requires extracting a two-dimensional table of data from a data source. If we wish to perform an analysis of countries and their economic indicators from the Gapminder data, for example to assess which countries are most similar to one another, we can construct a cross-sectional analysis population for a particular year. 

The Gapminder data table contains entries for same country for different years. The following SQL expression shows that 51 years are recorded in this version of the Gapminder data that we are using.

In [16]:
%sql SELECT COUNT (DISTINCT year) FROM gapminder_raw_data_table

,COUNT (DISTINCT year)
0,51


To select a cross-sectional subset of the Gapminder data, we simply use SQL to `SELECT` data where the year is 2002, as shown below. We store the results in a new table named `gapminder_2002_table` which can be the basis for a BayesDB analysis population.

In [17]:
%%sql
DROP TABLE IF EXISTS gapminder_2002_table;
CREATE TABLE gapminder_2002_table AS
    SELECT * FROM gapminder_raw_data_table
    WHERE "year" = 2002

""


As you can see below, the new table we have created retains one row per country. This is a useful population, because it can allow us to perform further analysis of which countries are most similar to one another using BayesDB's `SIMILARITY` query over the rows of a population. Depending on the research question of interest, it is possible to reshape the original data source into any number of two-dimensional tabular formats. For example, we can select data for a particular country, continent, or measurement type, or any combination. Alternatively, we can retain the entire original table and perform queries over a population of `"country-year"`'s. Some additional ways to construct populations are examined in subsequent sections of this tutorial.

In [18]:
%sql SELECT COUNT (DISTINCT country_id) FROM gapminder_2002_table

,COUNT (DISTINCT country_id)
0,170


In [19]:
%sql SELECT * FROM gapminder_2002_table;

,country_id,country,continent,year,aid_received,aid_received_log,invest_foreign_per1k,invest_foreign_per1k_log,invest_domestic_per1k,invest_domestic_per1k_log,spending_health_per1,spending_health_per1_log,gdp_per1k,mortality_kid,mortality_kid_log,life_expectancy,physicians,population,population_density,surface,gini,agriculture,hiv,malaria,professional_birth,sugar,food,infection,labor,broadband,cell_phone,internet,computers,it_tel,drought,drought_log,earthquake,earthquake_log,epidemic,epidemic_log,extreme_temp,extreme_temp_log,flood,flood_log,storm,storm_log,tsunami,tsunami_log,air_accident,air_accident_log,immune_diphtheria,immune_hepatitis,immune_hib,mcv,immune_tetanus,coal_per1,coal_per1_log,natural_gas_per1,natural_gas_per1_log,co2_per1,co2_per1_log,completion,energy_per1,energy_per1_log,sanitation,water_source,electricity_per1,electricity_per1_log,oil_per1,oil_per1_log,mortality_maternal,gdp_per1k_log,life_expectancy_log,mortality_maternal_log,delta_aid_received_log,delta_invest_foreign_per1k_log,delta_invest_domestic_per1k_log,delta_spending_health_per1_log,delta_gdp_per1k_log,delta_mortality_kid_log,delta_life_expectancy_log,delta_mortality_maternal_log,delta_aid_received_log_lag1,delta_aid_received_log_lag2,delta_aid_received_log_lag3,delta_aid_received_log_lag4,delta_invest_foreign_per1k_log_lag1,delta_invest_foreign_per1k_log_lag2,delta_invest_foreign_per1k_log_lag3,delta_invest_foreign_per1k_log_lag4,delta_invest_domestic_per1k_log_lag1,delta_invest_domestic_per1k_log_lag2,delta_invest_domestic_per1k_log_lag3,delta_invest_domestic_per1k_log_lag4,delta_spending_health_per1_log_lag1,delta_spending_health_per1_log_lag2,delta_spending_health_per1_log_lag3,delta_spending_health_per1_log_lag4
0,2,Afghanistan,Asia,2002,1906730000,9.280289,1.213439e-02,-1.915982,0.122530,-0.911759,14.818293,1.170798,1.053,130.3,2.114944,51.6,NaN,21487079,37.786,652230,NaN,57.882955,2200,2821.835960,NaN,NaN,NaN,76,59.799999,0.000000,0.101462,0.004092,NaN,0.261323,0,0.000000,102091,5.008987,209631,5.321456,0,0.000000,5626,3.750200,0,0.000000,NaN,NaN,0,0.000000,36,NaN,NaN,35,52,NaN,NaN,NaN,NaN,0.000017,-4.776680,NaN,NaN,NaN,33,30,NaN,NaN,NaN,NaN,532.566588,0.022428,1.712650,2.726374,0.432482,1.707231,NaN,NaN,0.086921,-0.011512,0.004229,NaN,8.847807,8.347779,8.321143,8.393119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,Albania,Europe,2002,457220000,8.660125,1.794084e-01,-0.746157,1.447918,0.160744,90.264318,1.955516,5.913,23.6,1.372912,74.5,1.305000,3123112,107.478,28750,28.15,41.605839,NaN,NaN,98.0,71.23,2864.93,10,66.300003,0.000000,27.542432,0.390081,1.17,34.574619,0,0.000000,0,0.000000,226,2.354108,0,0.000000,66885,4.825329,125006,5.096931,NaN,NaN,NaN,NaN,98,96,NaN,96,80,NaN,NaN,NaN,NaN,0.001200,-2.920865,NaN,0.649229,-0.187602,86,97,1459.433092,3.164184,NaN,NaN,82.410121,0.771808,1.872156,1.915981,0.025708,-0.209080,-0.037475,0.053175,0.013653,-0.023287,0.000583,0.102835,8.634417,8.698422,8.862251,8.592321,0.149533,0.543085,-0.093996,-0.062782,0.080207,0.146766,0.112374,0.041655,1.902341,1.876429,1.813074,1.673041
2,5,Algeria,Africa,2002,306980000,8.487110,1.985032e-01,-0.702233,3.316470,0.520676,69.924657,1.844630,10.634,37.8,1.577492,73.6,1.130000,31990387,13.201,2381740,NaN,16.733565,NaN,0.977273,95.9,82.19,3034.33,22,58.200001,0.000000,1.431990,1.591641,0.77,7.640696,0,0.000000,0,0.000000,0,0.000000,0,0.000000,2353,3.371622,0,0.000000,NaN,NaN,0,0.000000,86,NaN,NaN,81,66,0.019019,-1.720807,2.311812,0.363953,0.002840,-2.546717,NaN,0.916100,-0.038057,93,88,714.702866,2.854126,0.309905,-0.508771,10.959739,1.026697,1.866878,1.039800,-0.042487,-0.013671,0.075319,0.013311,0.017969,-0.012458,0.001182,-0.685011,8.529597,8.513604,8.318627,8.789686,0.607851,-0.065869,-0.314855,0.382678,0.052019,-0.053091,0.028337,0.097426,1.831319,1.796626,1.796029,1.819433
3,8,Angola,Africa,2002,609220000,8.784774,5.498199e-01,-0.259780,0.472482,-0.325615,18.149774,1.258871,3.759,211.7,2.325721,53.8,NaN,16109696,11.944,1246700,NaN,46.033529,140000,12639.2

### 4.5 Creating new indicator variables

Sometimes it is helpful as part of an analysis to create new variables derived from old ones. These can include different types of transformations: binary indicators, windowing or thresholding functions, lags, transformations into frequency domain, etc. Here we show how a simple type of indicator variable -- an indicator for missing data -- can be created and added to the data table and subsequently included as part of an analysis population.

The following expression creates indicator variables for "missingness". For a given variable, its "missingness indicator" takes a value of `1` where there is missing data, and `0` where there data is present. Using a `SQL` expression, we create a "missingness indicator" for a number of variables in the original data table.

In [20]:
%%sql
DROP TABLE IF EXISTS missing_table;

CREATE TABLE missing_table AS SELECT

    CASE WHEN "country_id"         IS NULL THEN 1 ELSE 0 END AS "missing_country_id",
    CASE WHEN "country"            IS NULL THEN 1 ELSE 0 END AS "missing_country",
    CASE WHEN "continent"          IS NULL THEN 1 ELSE 0 END AS "missing_continent",
    CASE WHEN "year"               IS NULL THEN 1 ELSE 0 END AS "missing_year",
    CASE WHEN "aid_received"       IS NULL THEN 1 ELSE 0 END AS "missing_aid_received",
    CASE WHEN "mortality_kid"      IS NULL THEN 1 ELSE 0 END AS "missing_mortality_kid",
    CASE WHEN "life_expectancy"    IS NULL THEN 1 ELSE 0 END AS "missing_life_expectancy",
    CASE WHEN "physicians"         IS NULL THEN 1 ELSE 0 END AS "missing_physicians",
    CASE WHEN "population"         IS NULL THEN 1 ELSE 0 END AS "missing_population",
    CASE WHEN "population_density" IS NULL THEN 1 ELSE 0 END AS "missing_population_density",
    CASE WHEN "surface"            IS NULL THEN 1 ELSE 0 END AS "missing_surface",
    CASE WHEN "gini"               IS NULL THEN 1 ELSE 0 END AS "missing_gini",
    CASE WHEN "agriculture"        IS NULL THEN 1 ELSE 0 END AS "missing_agriculture",
    CASE WHEN "hiv"                IS NULL THEN 1 ELSE 0 END AS "missing_hiv",
    CASE WHEN "malaria"            IS NULL THEN 1 ELSE 0 END AS "missing_malaria",
    CASE WHEN "professional_birth" IS NULL THEN 1 ELSE 0 END AS "missing_professional_birth",
    CASE WHEN "sugar"              IS NULL THEN 1 ELSE 0 END AS "missing_sugar",
    CASE WHEN "food"               IS NULL THEN 1 ELSE 0 END AS "missing_food",
    CASE WHEN "infection"          IS NULL THEN 1 ELSE 0 END AS "missing_infection",
    CASE WHEN "labor"              IS NULL THEN 1 ELSE 0 END AS "missing_labor",
    CASE WHEN "broadband"          IS NULL THEN 1 ELSE 0 END AS "missing_broadband",
    CASE WHEN "cell_phone"         IS NULL THEN 1 ELSE 0 END AS "missing_cell_phone",
    CASE WHEN "internet"           IS NULL THEN 1 ELSE 0 END AS "missing_internet",
    CASE WHEN "computers"          IS NULL THEN 1 ELSE 0 END AS "missing_computers",
    CASE WHEN "it_tel"             IS NULL THEN 1 ELSE 0 END AS "missing_it_tel",
    CASE WHEN "drought"            IS NULL THEN 1 ELSE 0 END AS "missing_drought",
    CASE WHEN "earthquake"         IS NULL THEN 1 ELSE 0 END AS "missing_earthquake"

FROM gapminder_2002_table

""


Now we display the results.

In [21]:
%sql SELECT * FROM missing_table LIMIT 5;

,missing_country_id,missing_country,missing_continent,missing_year,missing_aid_received,missing_mortality_kid,missing_life_expectancy,missing_physicians,missing_population,missing_population_density,missing_surface,missing_gini,missing_agriculture,missing_hiv,missing_malaria,missing_professional_birth,missing_sugar,missing_food,missing_infection,missing_labor,missing_broadband,missing_cell_phone,missing_internet,missing_computers,missing_it_tel,missing_drought,missing_earthquake
0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 4.6 Adding the new indicator variables to the data table

Now we add back the "missingness indicators" to the original table, and we save the results into a new table named `gapminder_2002_with_missing_indicators_table` that can be the basis for a BayesDB population.

In [22]:
%%sql
DROP TABLE IF EXISTS gapminder_2002_with_missing_indicators_table;

CREATE TABLE gapminder_2002_with_missing_indicators_table AS
    SELECT * FROM missing_table, gapminder_2002_table
    WHERE missing_table.rowid = gapminder_2002_table.rowid

""


We display some rows of the combined table to confirm that the indicator variables are correctly identifying missing data.

In [23]:
%sql SELECT missing_physicians, physicians FROM gapminder_2002_with_missing_indicators_table LIMIT 5;

,missing_physicians,physicians
0,1,NaN
1,0,1.305
2,0,1.130
3,1,NaN
4,1,NaN


### 4.7 Overriding BayesDB's default guesses for statistical types

We will now create BayesDB population from the 2002 Gapminder data table with the indicator variables added to it, `gapminder_2002_with_missing_indicators_table`. To create the population, we must first associate each column in the data table with a statistical type. We begin by checking whether BayesDB's guesses for statistical types are reasonable. We notice that some missingness indicators are ignored. 

In [24]:
%bql .guess_schema gapminder_2002_with_missing_indicators_table

MODEL 
	 "continent",
	 "missing_hiv",
	 "extreme_temp_log",
	 "earthquake_log",
	 "missing_professional_birth",
	 "missing_malaria",
	 "drought_log",
	 "earthquake",
	 "missing_drought",
	 "missing_gini",
	 "missing_earthquake",
	 "extreme_temp",
	 "missing_physicians",
	 "missing_aid_received",
	 "drought" 
AS 
	NOMINAL;
MODEL 
	 "epidemic_log",
	 "delta_aid_received_log_lag4",
	 "delta_aid_received_log_lag2",
	 "delta_aid_received_log_lag3",
	 "delta_aid_received_log_lag1",
	 "aid_received",
	 "mortality_kid_log",
	 "storm_log",
	 "population_density",
	 "delta_life_expectancy_log",
	 "life_expectancy_log",
	 "life_expectancy",
	 "delta_invest_foreign_per1k_log_lag3",
	 "delta_invest_foreign_per1k_log_lag4",
	 "delta_invest_foreign_per1k_log_lag1",
	 "sugar",
	 "physicians",
	 "gini",
	 "hiv",
	 "food",
	 "coal_per1_log",
	 "electricity_per1_log",
	 "delta_spending_health_per1_log_lag4",
	 "invest_foreign_per1k_log",
	 "delta_spending_health_per1_log_lag1",
	 "delta_spending_health_

Why has BayesDB ignored certain indicator variables? We can use the following expression to get more details. Some variables are ignored because the indicator takes only one value ("There is only one unique value"), which would occur when all or none of the data are missing. In that case, ignoring the indicator variable is a sensible choice, because it provides no information to distinguish any of the rows in the table.

However, some indicators are ignored because their values are mostly the same (fewer than 10% or more than 90% data missing). We would perfer to model those indicators as `NOMINAL` instead of ignoring them entirely. The next section will provide the syntax to create a population and override some of BayesDB's guessed stattypes.

In [25]:
%bql GUESS SCHEMA FOR gapminder_2002_with_missing_indicators_table

,column,stattype,num_distinct,reason
0,missing_country_id,ignore,1,There is only one unique value.
1,missing_country,ignore,1,There is only one unique value.
2,missing_continent,ignore,1,There is only one unique value.
3,missing_year,ignore,1,There is only one unique value.
4,missing_aid_received,nominal,2,There are fewer than 20 distinct numerical va...
5,missing_mortality_kid,ignore,1,There is only one unique value.
6,missing_life_expectancy,ignore,2,More than 90 percent of the values are the sa...
7,missing_physicians,nominal,2,There are fewer than 20 distinct numerical va...
8,missing_population,ignore,1,There is only one unique value.
9,missing_population_density,ignore,2,More than 90 percent of the values are the sa...


### 4.8 Creating the population

In the previous section, we identified certain statistical types that had been guessed by BayesDB that we wish to override. Below we create a BayesDB population and override some of the statistical types that BayesDB guesses. The statistical types of any variable *not* listed by name below are maintained as BayesDB's original guesses. 

In [26]:
%%mml
DROP POPULATION IF EXISTS gapminder_2002_with_missing_indicators_population;

CREATE POPULATION gapminder_2002_with_missing_indicators_population
    FOR gapminder_2002_with_missing_indicators_table WITH SCHEMA (

        GUESS STATTYPES OF (*);

        SET STATTYPES OF
            "missing_surface",
            "missing_food",
            "missing_agriculture",
            "missing_year",
            "missing_life_expectancy",
            "missing_cell_phone",
            "missing_labor",
            "tsunami_log",
            "missing_population",
            "missing_computers",
            "missing_sugar",
            "missing_internet",
            "missing_infection",
            "missing_population_density",
            "missing_broadband",
            "missing_it_tel"
        TO NOMINAL;
)

""


## 5. Assembling a population from multiple epidemiological data sources

In the fMRI data and Gapminder data examples that we have explored, data tables were `SELECT`ed from, joined, and altered (for example by adding indicator variables) using the `SQL` and `BQL` languges. Some users may wish to preprocess their data offline in Python, Matlab, or another language, then save the results as a `.csv` file to be uploaded to BayesDB. We demonstrate this workflow below.

### 5.1 Description of the data source

The data sources for this demonstration are related to tracking the spread of the influenza virus, or the flu. These include (a) the [Centers for Disease Control Outpatient Influenza-like Illness Surveillance Network](https://www.cdc.gov/flu/weekly/overview.htm) data; (b) state- and district-level weekly flu reports; (c) demographic data; (d) tweets on [Twitter](http://twitter.com) with flu-related keywords; (e) [weekly flu vaccine percentages data from Medicare](https://www.hhs.gov/nvpo/about/resources/interactive-mapping-tool-tracking-flu-vaccinations/index.html); and (f) geographic data. Populations derived from these data sources are intended to be used to predict the spread of the flu in the United States, and to model the effect of any interventions.

**Collaborators.** These data were provided to the Probabilistic Computing Group by DARPA as part of the [Probabilistic Programming for Advanced Machine Learning (PPAML)](http://www.darpa.mil/program/probabilistic-programming-for-advancing-machine-Learning) scientific program. The Python notebooks linked below were prepared by Feras Saad of Probcomp.

### 5.2 Creating a custom .csv file by using external Python notebooks

We link to separate notebooks for preprocessing and creating data tables based on the multiple time series within the flu prediction data sources:

* **Preprocessing:** The [preprocessing notebook](resources/preprocess_flu_data.ipynb) aggregates a number of different data files into a single `.json` file. Then, a user selects a set of states within the United States that are of interest for analysis. For each state, the`.json` records information about demographics, vaccinations, and flu-related tweets.


* **Creating data tables:** The [notebook to create data tables](resources/tabulate_flu_data.ipynb) processes this `.json` file further and saves tabular `.csv` files from the results. In that notebook, a user specifies a sliding window to extract a form of cross-sectional variables. 


Running these two external Python notebooks for pre-processing produces a number of new `.csv` files with sliding window style columns; this allows the the flu time series data to be analyzed using datatypes that are readily available in BayesDB. Feel free to explore these two linked notebooks to for an example of offline processing that includes a windowing function. The `.csv` files are loaded into BayesDB using the usual steps below.

### 5.3 Ingesting data from the .csv file

A `.csv` file is ingested in the usual manner, that we have demonstrated earlier in this tutorial.

In [27]:
%%bql
DROP TABLE IF EXISTS massachusetts_flu_vacc_table;
CREATE TABLE massachusetts_flu_vacc_table
    FROM 'resources/fludata/generated/tabulated_flu_vacc_ili_MA_lag5_censor.csv'

""


In [28]:
%sql SELECT * FROM massachusetts_flu_vacc_table LIMIT 5;

,tweets_t0,tweets_t1,tweets_t2,tweets_t3,tweets_t4,vaccs_t0,vaccs_t1,vaccs_t2,vaccs_t3,vaccs_t4,ili_t0,ili_t1,ili_t2,ili_t3,ili_t4
0,180,158,181,259,329,35.504888,76.986299,180.883452,355.892430,497.010113,45,53,38,26,43
1,158,181,259,329,432,76.986299,180.883452,355.892430,497.010113,619.861352,53,38,26,43,53
2,181,259,329,432,657,180.883452,355.892430,497.010113,619.861352,673.553593,38,26,43,53,50
3,259,329,432,657,708,355.892430,497.010113,619.861352,673.553593,647.302767,26,43,53,50,62
4,329,432,657,708,715,497.010113,619.861352,673.553593,647.302767,495.244267,43,53,50,62,70


### 5.4 Creating the population

A population is created in the usual manner.

In [29]:
%%mml
DROP POPULATION IF EXISTS massachusetts_flu_vacc_population;

CREATE POPULATION massachusetts_flu_vacc_population FOR massachusetts_flu_vacc_table
    WITH SCHEMA (GUESS STATTYPES OF (*));

""


## 6. Follow-up skills demonstrated in other notebooks

### 6.1 Modeling and querying the populations that you have created

Designing and assembling one or more populations for analysis is a core part of the process of using BayesDB. Indeed, compared with other analysis frameworks, BayesDB can be understood as an "artificially intelligent" data science assistant because one of its features is to automatically creates a probabilistic model just as soon as the user has defined any arbitrary **tabular, medium data** population. In BayesDB, this modeling is achieved in three (3) subsequent lines of code using a domain-general Bayesian modeling method called CrossCat, which is part of the Probcomp software stack. While BayesDB includes an "artificially intelligent" baseline probabilistic model, it also empowers a user to create arbitrary custom probabilistic models over the variables (columns) of a population. This is accomplished using the expressive language `VentureScript` that is part of Probcomp's software stack. There are countless possibilities for custom modeling, some of which include regressions, gaussian processes, random forests, etc. Please revisit [introduction.ipynb](introduction.ipynb) for a more detailed exposition.

In summary, after creating a population in BayesDB, the two steps that typically follow are:
* Modeling the population
* Performing queries based on the population and its probabilistic model

With a population and a model, the queries that may be performed immediately include: 
* estimating dependence probabilities or mutual information between variables
* estimating probabilistic similarity among rows (observations)
* identifying anomalous data
* simulating limitless numbers of data points (observed or unobserved in the original population)
* visualizing the data
* and much more. 

The set of exploratory and inferential data analysis possibilities in BayesDB are illustrated in the rest of this tutorial. The notebooks [gapminder-exploratory.ipynb](gapminder-exploratory.ipynb), [gapminder-missing-data.ipynb](gapminder-missing-data.ipynb), and [satellites-predictive.ipynb](satelittes-predictive.ipynb) demonstrate how to create a probabilistic model with using BayesDB's baseline method, CrossCat. The notebooks [hyperparameter-inference-with-gp.ipynb](hyperparameter-inference-with-gp.ipynb) and [extrapolation-with-gp.ipynb](extrapolation-with-gp.ipynb) demonstrate modeling with custom probabilistic models that a user defines in VentureScript.